In [1]:
from lxml import etree
import requests
from lxml.html import fromstring, tostring
import  jieba

In [2]:
def get_stock_news_url(url,news_url_all):    
    headers = {#输入headers
        'Accept': '*/*',
        'User - Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/66.0.3359.181 Safari/537.36'
    }
    ret = requests.get(url, headers=headers)#访问
    code = ret.apparent_encoding  # 获取url对应的编码格式
    ret.encoding = code           # 传入url对应的编码格式
    html = ret.text               # html文件内容即示例中的标签
    tree = etree.HTML(html)       #etree做html内容的整理，方便xpath定位
    result = tree.xpath('//*[@class="datelist"]//a//@href')#定位到新闻的herf
    title = tree.xpath('//*[@class="datelist"]//a/text()')#定位到新闻的herf
    for i in range(len(result)):
        news_url_all.append(result[i])#汇总到总的list
        title_all.append(title[i])

In [3]:
news_url_all = []
title_all = []
for page in range(5): #爬取前5页
    url = "https://vip.stock.finance.sina.com.cn/corp/view/vCB_AllNewsStock.php?symbol=sh603881&Page="+str(page+1)#数据港新闻URL
    get_stock_news_url(url,news_url_all)

In [7]:
news_url_all

['https://finance.sina.com.cn/stock/hyyj/2020-06-15/doc-iirczymk7068096.shtml',
 'https://finance.sina.com.cn/stock/relnews/cn/2020-06-09/doc-iirczymk6067576.shtml',
 'https://finance.sina.com.cn/stock/relnews/cn/2020-06-09/doc-iirczymk5990268.shtml',
 'https://finance.sina.com.cn/stock/relnews/cn/2020-06-09/doc-iirczymk6008990.shtml',
 'https://finance.sina.com.cn/stock/stockzmt/2020-06-08/doc-iirczymk5827605.shtml',
 'https://cj.sina.cn/articles/view/2868676035/aafc85c301900pb83',
 'https://finance.sina.com.cn/stock/cpbd/2020-06-08/doc-iirczymk5798844.shtml',
 'https://cj.sina.cn/articles/view/2868676035/aafc85c301900pb3q',
 'https://finance.sina.com.cn/stock/relnews/cn/2020-06-05/doc-iirczymk5409309.shtml',
 'https://finance.sina.cn/stock/ggdp/2020-06-03/detail-iirczymk5057673.d.html',
 'https://finance.sina.com.cn/7x24/2020-06-03/doc-iircuyvi6426423.shtml',
 'https://finance.sina.com.cn/roll/2020-06-03/doc-iircuyvi6425768.shtml',
 'https://finance.sina.com.cn/stock/stockzmt/2020-06

In [4]:
def deal_data(text):
    text =jieba.lcut(text);
    stopwords = {}.fromkeys([line.rstrip() for line in open('./停用词_中文.txt',encoding='UTF-8')]) #加载停用词(中文)
    eng_stopwords = set(stopwords) #去掉重复的词
    words = [w for w in text if w not in eng_stopwords] #去除文本中的停用词
    return ' '.join(words)

In [5]:
def analyse_news_urls(url,title):
    headers = {#输入headers
        'Accept': '*/*',
        'User - Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/66.0.3359.181 Safari/537.36'
    }
    ret = requests.get(url, headers=headers)#访问
    code = ret.apparent_encoding  # 获取url对应的编码格式
    ret.encoding = code           # 传入url对应的编码格式
    html = ret.text               # html文件内容即示例中的标签
    tree = etree.HTML(html)       #etree做html内容的整理，方便xpath定位
    single_news_list =[]
    try:
        news_date = tree.xpath('//span[@class="date"]/text()')[0] #定位获取新闻日期
    except:
        news_date = "抱歉，没有新闻日期"
    try:
        news_source = tree.xpath('//span[@class="source ent-source"]/text()')[0] #定位获取新闻来源
    except:
        news_source = "抱歉，没有新闻来源"
    content_list = tree.xpath('//p[@cms-style="font-L"]/text()') #定位获取新闻内容
    content = ""
    for i in content_list:
        content += i
    content = deal_data(content)
    single_news_list.append([title,news_date,url,news_source,content])
    return single_news_list

In [6]:
All_news_content = []
for i in range(len(news_url_all)):
    All_news_content.append(analyse_news_urls(news_url_all[i],title_all[i]))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Cappu\AppData\Local\Temp\jieba.cache
Loading model cost 2.192 seconds.
Prefix dict has been built successfully.


In [8]:
All_news_content

[[['国盛证券：光模块业绩确定 存在板块性配置机会',
   '2020年06月15日 11:17',
   'https://finance.sina.com.cn/stock/hyyj/2020-06-15/doc-iirczymk7068096.shtml',
   '新浪财经综合',
   '']],
 [['新基建之国内IDC：雁阵格局 新基建、新气象！',
   '2020年06月09日 14:28',
   'https://finance.sina.com.cn/stock/relnews/cn/2020-06-09/doc-iirczymk6067576.shtml',
   '海通证券股份有限公司',
   '全球 云 计算 5G   双轮 驱动 IDC   行业 步入 高 景气 2019Q2   亚马逊 微软 谷歌 三家 全球 北美 TOP3   云 计算 龙头企业 资本 开支 回暖 国内 阿里 腾讯 2019   年 资本 开支 同比 呈现 趋势 云 计算 行业 资本 开支 步入 新一轮 景气 周期 信通 院 云 计算 发展 白皮书 2019 2018   年 全球 云 计算 市场 规模 达 1363   亿美元 增速 23% 我国 公有 云 市场 规模 437.4   亿元 IaaS   规模 270.4   亿元 同比 增长 81.8% 占 公有 云 市场 61.8% 云 计算 发展 整体 正 处于 行业 上升期 IaaS   层 基础设施 建设 需求 旺盛 5G   带动 数据 流量 迎来 新一轮 爆发 期 国家 加码 新 基建 注入 新动力 华为 官网 华为 GIV   预计 2025   年 全球 人均 通信 流量 增长 10   倍 人均 日 通信 流量 达 4GB 人均 日 通信 流量 增长 约 30   倍 人均 日 通信 流量 达 1GB 5G   时期 流量 爆发式 增长 有望 带来 旺盛 IDC   市场需求 2020   年 国家 明确指出 加快 5G   网络 数据中心 新 基建 进度 赛迪 智库 电子信息 研究所 发布 新 基建 发展 白皮书 介绍 我国 2025   年 建成 数量 大型 超大型 数据中心 边缘 数据中心 海外 基础 电信 运营商 退出 第三方 IDC   主导 市场 国内 三大 电信 运营商 

In [173]:
file=open('Q2_Answer.txt','w',encoding='UTF-8') 
for i in All_news_content:
    file.write(str(i)+"\n")
file.close() 